In [15]:
import pandas as pd

In [16]:
columns = ['SID',
           'SEASON',
           'NUMBER',
           'BASIN',
           'SUBBASIN',
           'NAME',
           'ISO_TIME',
           'NATURE',
           'LAT',
           'LON',
           'WMO_WIND',
           'WMO_PRES',
           'TRACK_TYPE',
           'DIST2LAND',
           'LANDFALL',
           'USA_WIND',
           'USA_PRES',
           'STORM_SPEED',
           'STORM_DIR'
          ]

In [17]:
data = pd.read_csv('./data/ibtracs.SI.list.v04r00.csv', usecols=columns, dtype='str')

In [18]:
data.head()

,SID,SEASON,NUMBER,BASIN,SUBBASIN,NAME,ISO_TIME,NATURE,LAT,LON,WMO_WIND,WMO_PRES,TRACK_TYPE,DIST2LAND,LANDFALL,USA_WIND,USA_PRES,STORM_SPEED,STORM_DIR
0,,Year,,,,,,,degrees_north,degrees_east,kts,mb,,km,km,kts,mb,kts,degrees
1,1848011S09079,1848,1,SI,MM,NOT_NAMED,1848-01-11 06:00:00,NR,-9.00000,79.0000,,,spur-split,1679,1679,,,26,252
2,1848011S09079,1848,1,SI,MM,NOT_NAMED,1848-01-11 09:00:00,NR,-9.40782,77.7341,,,spur-split,1744,1744,,,26,252
3,1848011S09079,1848,1,SI,MM,NOT_NAMED,1848-01-11 12:00:00,NR,-9.83076,76.4735,,,spur-split,1817,1817,,,26,250
4,1848011S09079,1848,1,SI,MM,NOT_NAMED,1848-01-11 15:00:00,NR,-10.2882,75.2264,,,spur-split,1911,1911,,,26,248


The following block of code removes the record in 0, in data.head()

In [19]:
ibtracs_SI = data.drop(0, axis=0, inplace=False)
ibtracs_SI = ibtracs_SI.reset_index(inplace=False, drop=True)

In [20]:
ibtracs_SI.head()

,SID,SEASON,NUMBER,BASIN,SUBBASIN,NAME,ISO_TIME,NATURE,LAT,LON,WMO_WIND,WMO_PRES,TRACK_TYPE,DIST2LAND,LANDFALL,USA_WIND,USA_PRES,STORM_SPEED,STORM_DIR
0,1848011S09079,1848,1,SI,MM,NOT_NAMED,1848-01-11 06:00:00,NR,-9.00000,79.0000,,,spur-split,1679,1679,,,26,252
1,1848011S09079,1848,1,SI,MM,NOT_NAMED,1848-01-11 09:00:00,NR,-9.40782,77.7341,,,spur-split,1744,1744,,,26,252
2,1848011S09079,1848,1,SI,MM,NOT_NAMED,1848-01-11 12:00:00,NR,-9.83076,76.4735,,,spur-split,1817,1817,,,26,250
3,1848011S09079,1848,1,SI,MM,NOT_NAMED,1848-01-11 15:00:00,NR,-10.2882,75.2264,,,spur-split,1911,1911,,,26,248
4,1848011S09079,1848,1,SI,MM,NOT_NAMED,1848-01-11 18:00:00,NR,-10.8000,74.0000,,,spur-split,2007,1900,,,26,245


Let's do some queries, filter for year/season 2000

In [22]:
ibtracs_SI[ibtracs_SI['SEASON']==str(2000)]

,SID,SEASON,NUMBER,BASIN,SUBBASIN,NAME,ISO_TIME,NATURE,LAT,LON,WMO_WIND,WMO_PRES,TRACK_TYPE,DIST2LAND,LANDFALL,USA_WIND,USA_PRES,STORM_SPEED,STORM_DIR
135861,1999343S09095,2000,102,SI,WA,ILSA,1999-12-08 22:00:00,NR,-9.00000,95.0000,25,1002,main,924,914,,,5,98
135862,1999343S09095,2000,102,SI,WA,ILSA,1999-12-09 00:00:00,NR,-9.02366,95.1764,,,main,914,904,,,5,98
135863,1999343S09095,2000,102,SI,WA,ILSA,1999-12-09 03:00:00,NR,-9.05840,95.4417,,,main,914,900,,,5,97
135864,1999343S09095,2000,102,SI,WA,ILSA,1999-12-09 06:00:00,NR,-9.09142,95.7087,,,main,900,887,,,5,97
135865,1999343S09095,2000,102,SI,WA,ILSA,1999-12-09 09:00:00,NR,-9.12179,95.9783,,,main,887,878,,,5,96
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137957,2000105S09127,2000,28,SI,WA,ROSITA,2000-04-20 16:00:00,TS,-20.3966,127.303,35,995,main,0,0,33,,11,103
137958,2000105S09127,2000,28,SI,WA,ROSITA,2000-04-20 18:00:00,TS,-20.4763,127.674,,,main,0,0,30,,11,98
137959,2000105S09127,2000,28,SI,WA,ROSITA,2000-04-20 21:00:00,TS,-20.5295,128.258,,,main,0,0,30,,10,95
137960,2000105S09127,2000,28,SI,WA,ROSITA,2000-04-20 22:00:00,TS,-20.5331,128.382,30,1000,main,0,0,30,,3,78
